<a href="https://colab.research.google.com/github/W8andHope/scripts/blob/main/simula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats as st 
import random

In [ ]:
def current_milli_time():
    return round(time.thread_time() * 10000)

In [ ]:
n = 1000
values = []
#la semilla será dinámica, se usa el tiempo del sistema local 
#multiplicado por un número aleatorio de la librería random
ini = round(current_milli_time() * random.random())

#definimos la función del GLC
def glc(modulus, a, c, seed):
      global i 
      i = 0
      while i < n: 
        seed = (a * seed + c) % modulus
        values.append(round(seed/(modulus-1),6))
        i += 1

#introducimos los valores iniciales
glc(2**32,1103515245,12345,ini)
print("Números pseduoaleatorios: ",values)

#guardado de data en archivos .csv y .txt
name=str(ini)
df = pd.DataFrame({'':values}) 
df.to_csv('numeros.csv', index=False,sep=';',decimal='.')
df.to_csv('numeros.txt', index=False,sep=';',decimal='.')

#ploteo
plt.figure(figsize=(16,9))
plt.plot(range(500),values[0:500], marker=".", color="blue")
plt.show()

In [ ]:
#Minigenerador usando los mismo parámetros, este lo usaremos para la simulación
def mglc ():
  global a, c, modulus, seed
  seed = round(current_milli_time() * random.random())
  a = 1103515245
  c = 12345
  modulus = 2**32
  seed = (a * seed + c) % modulus
  return round(seed/(modulus-1),6)

In [ ]:
def simulacion(n):
  global sum1, sum2, p1, p2
  sum1 = 0
  sum2 = 0
  ncaja = 0
  cajas = list( {} for ncaja in range(n))
  while ncaja < n:
      #primer evento: selección de caja a inspeccionar
      r=mglc()
      if (r<=0.3):
        cajas[ncaja]["estado_caja"] = 1
      else:
        cajas[ncaja]["estado_caja"] = 0

      #segundo evento: cantidad de productos seleccionados a inspeccionar
      r=mglc()
      if (cajas[ncaja]["estado_caja"]==0):
        cajas[ncaja]["productos_seleccionados"]=0
      else: 
        if (r>0.8):
          cajas[ncaja]["productos_seleccionados"]=3
        else: 
          if (r>0.5):
            cajas[ncaja]["productos_seleccionados"]=2
          else:
            cajas[ncaja]["productos_seleccionados"]=1
      #tercer evento: estado de los productos de la caja
      r=mglc()
      if (r<=0.03):
        cajas[ncaja]["estado_producto"] = 1
      else:
        cajas[ncaja]["estado_producto"] = 0

      #cuarto evento: inspección de productos seleccionados : productos defectuosos encontrados
      r=mglc()
      if (cajas[ncaja]["estado_caja"]==0):
        cajas[ncaja]["productos_defectuosos_encontrados"] = 0
      else:
        if (cajas[ncaja]["estado_producto"] == 0):
          cajas[ncaja]["productos_defectuosos_encontrados"] = 0
        else: 
          if (r<=(cajas[ncaja]["productos_seleccionados"]*0.1)):
            cajas[ncaja]["productos_defectuosos_encontrados"] = 1
          else:
            cajas[ncaja]["productos_defectuosos_encontrados"] = 0
      sum1 = sum1 + cajas[ncaja]["productos_defectuosos_encontrados"]

      #cuarto evento: inspección de productos seleccionados : productos defectuosos no encontrados
      if (cajas[ncaja]["estado_producto"]==0):
        cajas[ncaja]["productos_defectuosos_no_encontrados"] = 0
      else:
        if (cajas[ncaja]["estado_caja"] == 0):
          cajas[ncaja]["productos_defectuosos_no_encontrados"] = 1
        else: 
          if (r<=(cajas[ncaja]["productos_seleccionados"]*0.1)):
            cajas[ncaja]["productos_defectuosos_no_encontrados"] = 0
          else:
            cajas[ncaja]["productos_defectuosos_no_encontrados"] = 1
      sum2 = sum2 + cajas[ncaja]["productos_defectuosos_no_encontrados"]

      #mostrar cajas 
      print("CAJA",ncaja+1,":",cajas[ncaja],"\n")
      ncaja += 1

  p1 = sum1 / n
  p2 = sum2 / n
  print("Probabilidad de encontrar una caja con productos defectuosos en la inspección: ",p1,"\n")
  print("Probabilidad de que una caja con productos defectuosos pase la inspección: ",p2)   


simulacion(1000)
